# RNA SEQ ANALYSIS

Parts of this code come from the sambomics tutorial: https://sanbomics.com/2022/01/08/complete-rnaseq-alignment-guide-from-fastq-to-count-table/

## 1º FASTQC

In [1]:
!fastqc -h


            FastQC - A high throughput sequence QC analysis tool

SYNOPSIS

	fastqc seqfile1 seqfile2 .. seqfileN

    fastqc [-o output dir] [--(no)extract] [-f fastq|bam|sam] 
           [-c contaminant file] seqfile1 .. seqfileN

DESCRIPTION

    FastQC reads a set of sequence files and produces from each one a quality
    control report consisting of a number of different modules, each one of 
    which will help to identify a different potential type of problem in your
    data.
    
    If no files to process are specified on the command line then the program
    will start as an interactive graphical application.  If files are provided
    on the command line then the program will run with no user interaction
    required.  In this mode it is suitable for inclusion into a standardised
    analysis pipeline.
    
    The options for the program as as follows:
    
    -h --help       Print this help file and exit
    
    -v --version    Print the version of the program and exit

The command used was:

fastqc --extract\
--delete\
--nogroup\
-o /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/fastqc\
/media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports//all_files/*.fq.gz

When the fastqc had run we could take a look to the output:

This are two plots from the report of a single file (as an example) but all of them looked like that:

- Here we can see that the quality is good along all the sequences and sequence position.


![](./images/PB_sequence_quality.png)


- Here we can see that the per base sequence content is normal along all the sequence until the 13th first bases, this deviation in the expected base frecquency suggest the pressence of adapters.


![](./images/PB_sequence_content.png)



- Thus, the next step will be a trimming with trim-galore. We will perform a hard timming to remove the adapters in the 13 fist positions. We wont conduct a quality or adapter trimming because we new that the quality is great. Also, we dont need to infer the adapters or to find them from a predefined adapter list, we can just remove the starting part of the sequences.

The command used was: 

trim_galore -o /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/trimmed_files\ # Output dir
            -j 8 \ # Number of cores recommended 
            --hardtrim3 137 \ # Length of the secuences after trimming (trim the first 13 bases)
            /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/all_files/*.fq.gz  # path to the fq files


First I had to create a new enviroment to run trim-galore due to dependencie issues:

conda create -n trimming trim-galore


**INDEX BUILDING:**

Then I had to build an index for the reference genome that STAR can use for the alignment.

- First I downloaded the FASTA and GTF files of the Human genome (GRCh38) from ensembl: https://www.ensembl.org/info/data/ftp/index.html

- Then I run the following command:

STAR --runMode genomeGenerate \
--genomeDir /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/GENOME_FILES/genome_index/ \
--genomeFastaFiles /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/GENOME_FILES/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa \
--sjdbGTFfile /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/GENOME_FILES/Homo_sapiens.GRCh38.110.gtf \
--runThreadN 30

**Actually perform the alignment:**

For the alignement I will take a manifest table (structure: [fq1 \t fq2 \t ID]) that contains the path info of the paired reads and a name for the sample, then I will iterate the rows of these table and run the STAR alignment command for the paired files of each row. 

- The command is:

STAR --runMode alignReads \
--runThreadN 30\
--genomeDir /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/GENOME_FILES/genome_index/ \
--outSAMtype BAM SortedByCoordinate \
--outFileNamePrefix {prefix}\
--readFilesIn {fq1} {fq2}
--readFilesCommand zcat


In [81]:
# Charge the manifest file used to indicate where are the paired reads 
import pandas as pd

manifest = pd.read_csv("./clean_data_reports/manifest.tsv",sep="\t",header=None)
manifest[0][0],manifest[1][0],manifest[2][0] # Show the row info

('/media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/trimmed_files/11_1.137bp_3prime.fq.gz',
 '/media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/trimmed_files/11_2.137bp_3prime.fq.gz',
 'ID: sample 11')

In [80]:
for i in range(len(manifest)):
    
    fq1 = manifest.iloc[i,0]
    fq2 = manifest.iloc[i,1]
    pref = manifest.iloc[i,2].replace("ID: ","").replace(" ","_")
    prefix= f"/media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/Alignment/{pref}"
    print(fq1,"\n",fq2,"\n",prefix,"\n","\n","\n",)
    !STAR --runMode alignReads --runThreadN 30 --genomeDir /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/GENOME_FILES/genome_index/ --outSAMtype BAM SortedByCoordinate --outFileNamePrefix {prefix} --readFilesIn {fq1} {fq2} --readFilesCommand zcat


/media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/trimmed_files/11_1.137bp_3prime.fq.gz 
 /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/trimmed_files/11_2.137bp_3prime.fq.gz 
 /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/Alignment/sample_11 
 
 

	/home/victor/miniconda3/envs/rnaseq/bin/STAR-avx2 --runMode alignReads --runThreadN 30 --genomeDir /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/GENOME_FILES/genome_index/ --outSAMtype BAM SortedByCoordinate --outFileNamePrefix /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/Alignment/sample_11 --readFilesIn /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/trimmed_files/11_1.137bp_3prime.fq.gz /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/trimmed_files/

**FEATURE COUNTS:**

Finally we will run featurecounts that will output a counts table which will be the starting point for the downstream analysis (differential expression, functional analysis...)

- Installation: 

conda install -c bioconda subread

- Command:

featureCounts -a /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/GENOME_FILES/Homo_sapiens.GRCh38.110.gtf \
-o /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/featurecounts/counts.tsv \
-T 30 -p -B -C /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/Alignment/*.bam


In [83]:
! featureCounts -h

featureCounts: invalid option -- 'h'

Version 2.0.6

Usage: featureCounts [options] -a <annotation_file> -o <output_file> input_file1 [input_file2] ... 

## Mandatory arguments:

  -a <string>         Name of an annotation file. GTF/GFF format by default. See
                      -F option for more format information. Inbuilt annotations
                      (SAF format) is available in 'annotation' directory of the
                      package. Gzipped file is also accepted.

  -o <string>         Name of output file including read counts. A separate file
                      including summary statistics of counting results is also
                      included in the output ('<string>.summary'). Both files
                      are in tab delimited format.

  input_file1 [input_file2] ...   A list of SAM or BAM format files. They can be
                      either name or location sorted. If no files provided,
                      <stdin> input is expected. Location-sorted pai

Finally we generate reports for all the steps with multiqc:

multiqc /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/Alignment /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/featurecounts /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/fastqc /media/victor/c1d5c312-b546-4d5e-b24f-72dbe9e6f18f/SRR_files/Velasco_et_al/clean_data_reports/trimmed_files
